# Quick Start

This notebook is intended to be a quick overview on Labelbox-Python SDK by demonstrating a simple but common work flow

In this guide, we will be:

1. Creating a dataset:
 A dataset is a collection of data rows imported into Labelbox.
2. Importing an image data row
3. Creating an ontology
4. Creating an project and attaching an ontology
5. Sending our data row towards are project by creating a batch
6. Exporting from our project

This notebook is geared towards new users of our SDK.

## Setup

To start, we first need to install the labelbox library and then import the SDK module. It is recommended to install "labelbox[data]" over labelbox to obtain all the correct dependencies. We will also be importing the python UUID library to generate unique IDs for the variety of objects that will be created with this notebook.

In [ ]:
%pip install -q "labelbox[data]"

In [ ]:
import labelbox as lb
import uuid

## API Key and Client
Provide a valid API key below to connect to the Labelbox client properly. For more information, please review the [Create API Key](https://docs.labelbox.com/reference/create-api-key) guide.

In [ ]:
API_KEY = None
client = lb.Client(api_key=API_KEY)